In [1]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.datasets import make_friedman1
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from xgboost.sklearn import XGBClassifier, XGBRegressor
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.linear_model import ElasticNet, LinearRegression
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from lightgbm import LGBMRegressor, LGBMClassifier
from concurrent.futures import ThreadPoolExecutor


In [148]:
ESTIMATOR_REG = [RandomForestRegressor(), KNeighborsRegressor(), SVR(), 
                XGBRegressor(), LGBMRegressor(), DecisionTreeRegressor()]
                
ESTIMATOR_CLF = [RandomForestClassifier(), SVC(), KNeighborsClassifier(), 
                XGBClassifier(), DecisionTreeClassifier(), LGBMClassifier(), 
                DecisionTreeClassifier()]


In [178]:
class meta_learner(BaseEstimator):
    """A meta-learner class that combines base estimators using a meta estimator.

    Parameters:
    -----------
    base_estimators : list
        A list of base estimators to be used.
    meta_estimator : object
        The meta estimator used to combine the predictions of the base estimators.

    Methods:
    --------
    fit(X, y):
        Fit the base estimators on the training data and target.
    predict(X):
        Predict the target values for the input data using the base and meta estimators.

    """

    def __init__(self, base_estimators, meta_estimator):
        self.base_estimators = base_estimators
        self.meta_estimator = meta_estimator

    def fit(self, X, y):
        """Fit the base estimators on the training data and target.

        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            The training data.
        y : array-like, shape (n_samples,)
            The target values.

        """
        def model_fit(model): return model.fit(X, y)
        with ThreadPoolExecutor(max_workers=len(self.base_estimators)) as executor:
            res = executor.map(model_fit, self.base_estimators)
            self.base_estimators = [i for i in res]

        base_preds = []
        self.X = X
        with ThreadPoolExecutor(max_workers=len(self.base_estimators)) as executor:
            res = executor.map(self.__model_predict, self.base_estimators)
            base_preds = np.column_stack([i for i in res])
        self.meta_estimator.fit(base_preds, y)

    def predict(self, X):
        """Predict the target values for the input data using the base and meta estimators.

        Parameters:
        -----------
        X : array-like, shape (n_samples, n_features)
            The input data.

        Returns:
        --------
        y_pred : array-like, shape (n_samples,)
            The predicted target values.

        """
        X = check_array(X)
        self.X = X
        base_preds = []
        with ThreadPoolExecutor(max_workers=len(self.base_estimators)) as executor:
            res = executor.map(self.__model_predict, self.base_estimators)
            base_preds = np.column_stack([i for i in res])
        return self.meta_estimator.predict(base_preds)

    def __model_predict(self, model): return model.predict(self.X)


In [179]:
X, y = make_friedman1()
X_train, X_test, y_train, y_test = train_test_split(X, y)


In [180]:
meta = meta_learner(ESTIMATOR_REG, LinearRegression())


In [182]:
meta.fit(X_train, y_train)

In [183]:
meta.predict(X_train)

array([14.98640234, 12.22423747,  9.7409516 , 16.87626314, 17.89585878,
       15.70180884, 12.71914375,  5.09494379, 12.26302873, 20.86818582,
       12.81334597, 14.15205596, 14.53409368, 16.42654087, 11.92127748,
       14.64970392, 19.25482772, 12.90357584, 16.12821663,  9.55050898,
       11.85588787, 15.44540672, 21.83313456, 20.9569284 , 19.44162237,
       12.84176221, 14.37125123, 17.01663691, 13.01206988, 11.91930039,
       12.27392707,  9.17040802, 15.77414345, 12.79081115, 16.54228914,
       15.01446276,  7.51271042, 18.61426757, 15.6622939 ,  7.120763  ,
       13.45329758,  4.93159591, 11.32958514, 11.29093078,  7.84406661,
       14.95468859, 16.63172048, 16.32097904,  4.67406812, 20.36020593,
       10.4499567 , 19.81281756, 18.53740983, 12.39453249,  7.8440786 ,
        6.61750105, 15.66431069, 19.52029907, 15.4027207 , 20.66326759,
       21.72107345,  5.39486388, 12.19208617, 18.70735457, 12.01215677,
       13.88225043, 10.25767762, 20.51586405, 11.83340492, 12.08